## Encoder - Decoder
现有的预训练模型有 **只基于Encoder**的BERT 和 **只基于Decoder**的GPT 
### 1.1 Seq2Seq
**Seq2Seq**，即序列到序列，即模型输入的是一个自然语言序列，输出是一个可能不等长的自然语言序列。
- 典型应用：机器翻译、文本摘要、对话系统等 几乎所有的NLP任务都可以视为Seq2Seq任务

在Seq2Seq任务中，一般思路为：
- **编码**，将输入的自然语言序列通过隐藏层编码为能够表征语义的向量
- **解码**，将向量通过隐藏层输出，再解码为自然语言序列。

**Transformer中的Encoder -> 编码 ，Decoder -> 解码** ？？？
### 1.2 前馈神经网络
前馈神经网络网络的特点是：
- 信息流是单向传播的
- 最重要的是有**激活函数**来保证前馈神经网络能够拟合任意函数形式，而不是**线性回归的堆叠**

In [1]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,resid_pdrop=0.1):
        super(MLP,self).__init__()
        self.linear1 = nn.Linear(input_size,hidden_size)
        self.linear2 = nn.Linear(hidden_size,output_size)
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(resid_pdrop)
    def forward(self,x):
        return self.dropout(self.linear2(self.activation(self.linear1(x))))
 #该前馈神经网络是由两个线性层中间加一个 GELU 激活函数组成的，以及前馈神经网络还加入了一个 Dropout 层来防止过拟合       


## 1.3 层归一化 (LayerNorm)
归⼀化核⼼是为了让不同层输⼊的取值范围或者分布能够⽐较⼀致。主流归一化有两种：批归一化(batch Norm)和层归一化(Layer Norm)

批归一化 是 计算一个batch_size的样本中的 每个特征维度d上的均值和方差

层归一化是计算每一个token中的特征维度的均值和方差，只看属于自己的向量

In [2]:
import torch.nn as nn 
class LayerNorm(nn.Module):
    def __init__(self,size,eps=1e-6):
        super(LayerNorm,self).__init__()
        self.eps=eps
        self.a = nn.Parameter(torch.ones(size))
        self.b = nn.Parameter(torch.zeros(size))#可学习参数
    def forward(self,x):
        mean =x.mean(-1,keepdim=True) # x的维度为B，T，C
        #mean的维度为B，T，1
        std =x.std(-1,keepdim=True)
        #std的维度亦为B，T，1
        return self.a*(x-mean)/(std+self.eps)+self.b
    


## 1.4 残差连接
由于Transformer模型的结果较复杂，为了避免模型退化，减少梯度消失的情况，引入了残差连接的思想。
即将上一层的输入与下一层的输出结合起来，以允许最底层的信息传递到最高层

同时注意在每进入一个模块时都需要对数据进行 归一化和残差连接

## 1.5 Encoder
Encoder 由 N 个 Encoder Layer 组成，
每⼀个 Encoder Layer 包括⼀个注意⼒层和⼀个前馈神经⽹络。

In [3]:
import torch.nn as nn
class EncoderLayer(nn.Module):
    '''Encoder layer block'''
    def __init__(self,n_emdb,n_head,hidden_size,attn_pdrop=0.1,resid_pdrop=0.1):
        super(EncoderLayer,self).__init__()
        self.ln_1 = LayerNorm(n_emdb)
        self.ln_2 = LayerNorm(n_emdb)
        self.attn = MultiHeadAttention(n_emdb,n_head,attn_pdrop,resid_pdrop)
        self.mlp = MLP(n_emdb,hidden_size,n_emdb,resid_pdrop)
    def forward(self,x,mask=None):
        x = x +self.attn(self.ln_1(x),mask=mask)
        x= x + self.mlp(self.ln_2(x))
        return x 
    

In [4]:
import torch.nn as nn
class EncoderBlock(nn.Module):
    def __init__(self,n_emdb,n_head,n_layer,hidden_size,attn_pdrop=0.1,resid_pdrop=0.1):
        '''Encoder block'''
        super(EncoderBlock,self).__init__()
        self.layers = nn.ModuleList(
            [EncoderLayer(n_emdb,n_head,hidden_size,attn_pdrop,resid_pdrop) for _ in range(n_layer)]
        )
        self.ln_f = LayerNorm(n_emdb)
    def forward(self,x,mask=None):
        for layer in self.layers:
            x= layer(x,mask=mask)
        return self.ln_f(x)

## 1.6 Decoder
Decoder包含两个注意力层
第⼀个注意⼒层是⼀个掩码⾃注意⼒层，即使⽤ Mask 的注意⼒计算，保证每⼀个 token 只能使⽤该 token 之前的注意⼒分数；
第⼆个注意⼒层是⼀个多头注意⼒层，该层将使⽤第⼀个注意⼒层的输出作为 query，使⽤ Encoder 的输出作为 key 和 value，来计算注意⼒分数

我发现原来写的MultiHeadAttention并不能与上面的要求接口相对齐，所以实现细节和实现接口先不管了

In [11]:
import torch.nn as nn
class DecoderLayer(nn.Module):
    '''Decoder layer block'''
    def __init__(self,n_embd,n_head,hidden_size,attn_pdrop=0.1,resid_pdrop=0.1):
        super(DecoderLayer,self).__init__()
        self.ln_1 = LayerNorm(n_embd)
        self.ln_2 = LayerNorm(n_embd)
        self.ln_3 = LayerNorm(n_embd)
        self.self_attn = MultiHeadAttention(n_embd,n_head,attn_pdrop,resid_pdrop)
        self.cross_attn = MultiHeadAttention(n_embd,n_head,attn_pdrop,resid_pdrop)
        self.mlp = MLP(n_embd,hidden_size,n_embd,resid_pdrop)
    def forward(self,x,enc_out,cross_attn_mask=None,attn_mask=None):
        x= x + self.self_attn(self.ln_1(x),mask=attn_mask)
        x = x + self.cross_attn(self.ln_2(x),kv = enc_out,mask = cross_attn_mask)
        x = x + self.mlp(self.ln_3(x))
        return x

In [12]:
import torch.nn as nn
class DecoderBlock(nn.Module):
    def __init__(self,n_embd,n_head,n_layer,hidden_size,attn_pdrop=0.1,resid_pdrop=0.1):
        '''Decoder block'''
        super(DecoderBlock,self).__init__()
        self.layers = nn.ModuleList(
            [DecoderLayer(n_embd,n_head,hidden_size,attn_pdrop,resid_pdrop) for _ in range(n_layer)]
        )
        self.ln_f = LayerNorm(n_embd)
    def forward(self,x,enc_out,cross_attn_mask=None,attn_mask=None):
        for layer in self.layers:
            x= layer(x,enc_out,cross_attn_mask,attn_mask)
        return self.ln_f(x)
    #该解码器块由多个解码器层组成，每个解码器层包含自注意力机制、交叉注意力机制和前馈神经网络，并在每个子层之后应用层归一化

**可以看出上述的encoder与decoder的融合是按这张图片的逻辑**
![可以看出上述的encoder与decoder的融合是按这张图片的逻辑](../imgae/2-0.jpg)